# 0 - Package Initialization

In [44]:
#Python version used: 3.9.12
#Packages necessary to construct a kinase network from the user uploaded file
import pandas as pd
import networkx as nx
from networkx import *
import csv
import itertools
from itertools import chain
import numpy as np
from scipy.stats import norm
import statistics
from statistics import *
import operator
import functools
#Visualizations
import plotly.express as px
import plotly.graph_objects as go 
from pyvis.network import Network
#Community detection
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sknetwork.clustering import Louvain
import collections
from collections import ChainMap

# 1 - Functions

## 1.1 - Extracting information from a z-scores dataset

In [2]:
# input: dataset must be a csv files in which the first column is the kinase interactions (pairs), and each following 
# column the z-scores of each of the kinase interactions in one of the treatments
# output: treatments are the headers of each of the z-scores columns, used to name visulizations returned from the analysis
# negative_edges is a list of lists, each list corresponding to a treatment in the dataset, it's contents being the 
# kinase interactions in said treatment with a z_score <0 and their z_scores,in the shape of tuples, i.e. a network
# negative_nodes is a list of lists, each list corresponding to the kinases of one network
def dataset_info(dataset, rem_vs_control='no', sign = 'neg'):
    # Read dataset and drop rows with no z-score information
    networkfile = pd.read_csv(dataset)
    networkfile = networkfile.dropna()
    
    # Return the headers of the z_scores columns, i.e. the treatments names
    headers = list(networkfile.columns.values)[1:]
    treatments = [x.split(".")[-1] for x in headers]
    if rem_vs_control == 'yes':
        treatments = [x.split("_")[0] for x in treatments]
    
    # Convert first column (kinase interactions/pairs) of the document to a list
    edges_list= networkfile[networkfile.columns[0]].to_list()
    # Convert the list into a list of tuples, each tuple being one kinase pair/edge (since this is the format needed to 
    # add edges onto pyvis.network (network visualization package))
    edges_list2 = [tuple(x.split('.',1)) for x in edges_list]
    
    #For each treatment(column) on the file, store kinase interactions/pairs with a z-score <0, and their z-scores
    if sign == 'neg':
        edges = [[(*edges_list2[n],+ networkfile[networkfile.columns[na]].to_list()[n]) for n in range(len(edges_list2)) if networkfile[networkfile.columns[na]].to_list()[n] < 0 ] for na in range(1,len(headers))]
    elif sign == 'pos' :
        edges = [[(*edges_list2[n],+ networkfile[networkfile.columns[na]].to_list()[n]) for n in range(len(edges_list2)) if networkfile[networkfile.columns[na]].to_list()[n] > 0 ] for na in range(1,len(headers))]
    
    #Kinases in each of the negative_edges lists
    nodes = []
    for selected_treatment in edges:
        #Keep only the kinases names (i.e. remove the z-scores) from the list of edges
        nodes_bf = [ (a,b) for a,b,c in selected_treatment]
        #Flatten list of edges tuples to list of nodes
        nodes = list(itertools.chain(*nodes_bf))
        #Remove duplicates from the list
        nodes_list = list(dict.fromkeys(nodes))
        nodes.append(nodes_list)
        
    return treatments,edges,nodes
 

## 1.2 - Community detection

In [ ]:
# 1. Are all the final runs the same?

In [60]:
# input: edges_list is a list corresponding to a treatment in the dataset, it's contents being the kinase interactions
# in said treatment with a z_score <0 and their z_scores,in the shape of tuples, i.e. a network 
# output: A_asso is the converged association matrix, n_iter the number of iterations required for convergence and 
# labels are the converged community labels 
def converge_asso(edges_list):
    
    #Initialize a Networkx instance
    negativenetwork = nx.Graph()
    #Add the kinase interactions & z-scores of treatment w to the network
    negativenetwork.add_weighted_edges_from(edges_list)
    #Store the individual kinases in the network as a numpy array - to return kinases in each community at the end
    node_0 = list(negativenetwork.nodes())
    node_0_arr = np.asarray(node_0)
    #A_0 is the adjacency matrix of the matrix (network instance) stored as a numpy array
    A_0 = to_numpy_array(negativenetwork)
    
    n_runs = 100
    A_runs = np.zeros((n_runs, len(A_0)))
    #Run the community detection algorithm on the network A_0 an n_runs amount of times
    for i in range(n_runs):
        #Louvain algorithm with Potts null network
        louvain = Louvain(1, 'potts', shuffle_nodes = 1)
        #Communities identified by number (Community 1, Community 2...)
        #Array is the length of the number of kinases in the network
        #For each kinase, it's community assignement is indicated by number
        labels = louvain.fit_transform(A_0)
        #Store the community partitions of each run
        A_runs[i,:] = labels

    #X = Number of kinases in the network
    #Create & store 'X' arrays of 'X' zeros each
    A_asso = np.zeros((len(A_0), len(A_0)))
    #After loop, each array corresponds to one kinase, and each position within an array indicates in how many runs that kinase is
    #assigned to the same community as each of the kinases in the network 
    #Loop over each run
    for i in range(n_runs):
        #Loop over each kinase/node in network (identified by number)
        for j in range(len(A_0)):
            #Loop over each kinase/node in network (identified by number)
            for l in range(len(A_0)):
            #If j and l are assigned to the same community in run i, add '1' to position l of array j   
                A_asso[j,l] = A_asso[j,l] + (A_runs[i,j] == A_runs[i,l])
    
    #Divide each element in each of the arrays in the converged association matrix(A_asso) by n_runs to get
    #the percentage (in 0.1(=10%) format) of runs in which each kinase-kinase/node-node combination is assigned to the same community
    A_asso = np.true_divide(A_asso, n_runs)
    
    n_iter = 0
    #until all the entries in A_asso are 0(not grouped together) or 1(grouped together) (all runs are the same)
    while(len(np.unique(A_asso)) != 2):
        n_runs = 100
        #Create an array for each run, the length of each array being the number of kinases in the network
        A_runs = np.zeros((n_runs, len(A_0)))
        #Repeat community detection on co-classification matrix (the weight of the edges is the percentage of runs in which two kinases are grouped together)
        for i in range(n_runs):
            louvain = Louvain(1, 'potts', shuffle_nodes = 1)
            labels = louvain.fit_transform(A_asso)
            A_runs[i,:] = labels
        A_asso = np.zeros((len(A_0), len(A_0)))
        for i in range(n_runs):
            for j in range(len(A_0)):
                for l in range(len(A_0)):
                    A_asso[j,l] = A_asso[j,l] + (A_runs[i,j] == A_runs[i,l])
        #Return percentage of runs in which each kinase-kinase combination is a ssigned to the same community
        A_asso = np.true_divide(A_asso, n_runs)
        n_iter = n_iter + 1 
    
    #Final community partitions (all runs are the same)
    labels = A_runs[0,:]
    
    #find how many communities are identified
    n_of_com = list(set(labels.flatten()))

    #List of communities of kinases as lists
    communities_list = [list(itertools.chain(*node_0_arr[np.argwhere(labels == int(n))])) for n in n_of_com]
    #Dictionaries of communities as networkx instances
    communities_nx = [dict.fromkeys(com,negativenetwork.subgraph(com)) for com in communities_list]
    communities_nx = dict(ChainMap(*communities_nx))
    
    return communities_list, communities_nx


## 1.3 - Reproducibility check

In [346]:
#Run converge_asso an n_checks number of times and check if the contents of the community of interest differ
def rep_check(n_checks, edges_list, target):
    community = ['a'] * n_checks
    for n in range(n_checks):
        #Store contents of community of interest
        community[n] = [com for com in converge_asso(edges_list)[0] if target in com]
    #Check if there are any kinases that are not present in all the community versions
    all_com = list(itertools.chain(*community))
    communities_list_rem = [x for x in set(all_com) if all_com.count(x)==n_checks]
    #Dictionary of networkx instances
    communities_nx_rem
    return communities_list_rem, communities_nx_rem


## 1.4 - Community strength calculation & display in heatmap

In [6]:
# input: edges_list is a list corresponding to a treatment in the dataset, it's contents being the kinase interactions
# in said treatment with a z_score <0 and their z_scores,in the shape of tuples, i.e. a network
# community is a list of the kinases in the community of interest
# output: dictionary of kinases and their community strength
def cs_calculation(treat_int, edges, treatments, targets, communities, community_edges, sign='−'):
    #Calculate the community strength of each of the kinases in the two selected communities you are comparing
    kin_cs = ['a'] * len(treat_int)
    for n in range(len(treat_int)):
        na = treat_int[n]
        # Store kinase pairs in which both kinases are in the community of interest
        #community_edges = [x for x in edges[na] if x[0] in communities[na] and x[1] in communities[na]]
        # Calculate the community strength of each of the kinases in the community and store as dictionary
        kin_cs[n] = {kin:abs(round(sum([community_edges[n][2] for n in range(len(community_edges)) if community_edges[n][1]==kin or community_edges[n][0]==kin]), 2)) for kin in communities[na]}    
    
    #Combine two communities lists
    kinases = list(dict.fromkeys(communities[treat_int[0]]+communities[treat_int[1]]))
    #Get values for each of the kinases in the combined list in each of the communities
    #If kinase is not in the community then the value is None
    values = [[kin_cs[n][kin] if kin in communities[treat_int[n]] else None for kin in kinases]for n in [0,1]]
    
    #Define heatmap axis and text
    x = kinases
    #<sub></sub> for subscript, <sup></sup>
    y = [ treatments[na]+'<sup>'+sign+'</sup>'+'<sub>('+targets[na]+')</sub>'+'<br>community strength</br>' for na in treat_int]
    z = [values[0],values[1]]
    
    #kinases, treatment,  = list(zip(x, y, z))
    
    return [x,y,z]


# 2 - Example 1 - Analysis and Results

# Negative edges

In [4]:
#Store dataset treatments names, kinase interactions (network edges) with a z-score < 0, and their z-scores (edges weight) 
[treatments, negative_edges, negative_nodes] = dataset_info('input/PROJECT_DATASET_2.csv', rem_vs_control='yes')
#Main kinase targets of each of the treatments/inhibitors in the dataset
targets = ['MAP2K1','PIK3CA','AKT1_2','MAPK1_3']

treatments

['Trametinib',
 'GDC0941',
 'AZD5363',
 'GDC0994',
 'LY2835219',
 'AZD5438',
 'CFI402257']

## 2.1 - Selected communities contents for networks trametinib(MAP2K1)[-], GDC0994(MAPK1_3), GDC0941(PI3K)[-], and AZD5363(AKT)[-]

In [ ]:
#Please note that the following block of code takes a few minutes to run

In [351]:
#Store contents of communities identified in each network
all_communities = ['a'] * len(targets)
all_communities_edges = ['a'] * len(targets)
all_kin_no_com = ['a'] * len(targets)
all_kin_no_com_edges = ['a'] * len(targets)
target_communities = ['a'] * len(targets)
target_communities_edges = ['a'] * len(targets)

for n in list(range(0,len(targets))):
    [kin_communities, kin_communities_edges, kin_no_com, kin_no_com_edges, target_community, target_community_edges] = converge_asso(negative_edges[n], target = targets[n])
    #All identified communities
    all_communities[n] = kin_communities
    all_communities_edges[n] = kin_communities_edges
    all_kin_no_com[n] = kin_no_com
    all_kin_no_com_edges[n] = kin_no_com_edges
    #Community containing the target
    target_communities[n] = target_community
    target_communities_edges[n] = target_community_edges

#Because we noticed that when extracting the community of interest from the trametinib(MEK)[-] network with the converge_asso
#function the contents of said community differ from run from run, for this network we run the community detection function (converge_asso)
#50 times, and keep only the nodes that are present in the community in all 50 runs of the converge_asso function
[target_communities[0],target_communities_edges[0]] = rep_check(50, negative_edges[0], targets[0])

print(all_communities[0])
print(all_kin_no_com_edges[0])



[['CAMKK2', 'CDK9', 'ERN1', 'IRAK1', 'CAMK2D', 'CIT', 'MAST1', 'ABL2', 'TAOK3', 'CDK6', 'CDK1', 'CDK5', 'MAPK14', 'CDK2', 'CAMK2G', 'CAMK2A', 'PAK2', 'DYRK2', 'PHKG2', 'MAP4K4', 'MAP3K15', 'CSNK2A2', 'CDKL5', 'ULK1', 'CDK4', 'RPS6KA4', 'EIF2AK1', 'TAOK1', 'CDK17', 'SIK2'], ['PAK1', 'PAK3', 'PIK3CA', 'TTK', 'PLK1', 'PIK3CB', 'LATS1', 'PRKCI', 'TNK2', 'MAP3K1', 'MTOR', 'YES1', 'AKT1_2', 'PDGFRB', 'CSNK1E', 'MAP2K1', 'MAPK1_3', 'RPS6KA2', 'STK3', 'PRKACA', 'PAK4', 'STK4', 'MINK1', 'PRKACB', 'RPS6KB1'], ['ABL1', 'LIMK1_2', 'SRPK1', 'SRPK3'], ['RPS6KA3', 'MAP3K3'], ['CLK1', 'CLK2']]
[[('ARAF', 'IRAK1', -0.014783465), ('ARAF', 'CDK9', -0.128200402), ('ARAF', 'MAP4K4', -0.125064244), ('ABL2', 'ARAF', -0.013631691), ('ARAF', 'CDK4', -0.086056068), ('ARAF', 'PIK3CB', -0.1020479), ('ARAF', 'CDK1', -0.107263093), ('ARAF', 'CDK5', -0.107263093), ('ARAF', 'MAST1', -0.03202431), ('AKT1_2', 'ARAF', -0.130900572), ('ARAF', 'CDK6', -0.227400209), ('ARAF', 'LATS1', -0.158995141), ('ARAF', 'MAP2K1', -0.1

In [349]:
print(all_kin_no_com[0])
print(all_kin_no_com_edges[0])
negative_edges[0]

[['MAPKAPK2'], ['RPS6KA6'], ['MARK3'], ['MAP4K5'], ['CDK7']]
[[], [], [], [], []]


[('CAMKK2', 'CDK9', -0.03059148),
 ('ERN1', 'IRAK1', -0.230053094),
 ('CAMK2D', 'CAMKK2', -0.056016515),
 ('ABL1', 'LIMK1_2', -0.022289867),
 ('PAK1', 'PAK3', -0.145222545),
 ('CIT', 'MAST1', -0.369626651),
 ('CAMK2D', 'CIT', -0.080753734),
 ('ABL1', 'ABL2', -0.021507521),
 ('ABL2', 'LIMK1_2', -0.031773846),
 ('CAMKK2', 'MAST1', -0.425601923),
 ('MAST1', 'TAOK3', -0.479000632),
 ('CDK6', 'CDK9', -0.37606021),
 ('CAMK2D', 'TAOK3', -0.052902013),
 ('CAMK2D', 'CDK9', -0.07834263),
 ('CAMKK2', 'CDK6', -0.19901646),
 ('CDK6', 'TAOK3', -0.179054584),
 ('CDK6', 'CIT', -0.192891404),
 ('ARAF', 'IRAK1', -0.014783465),
 ('CDK1', 'CDK5', -0.498872382),
 ('IRAK1', 'MAST1', -0.41758307),
 ('CDK6', 'IRAK1', -0.172504063),
 ('PIK3CA', 'TTK', -0.228672557),
 ('MAPK14', 'PLK1', -0.219707624),
 ('CDK1', 'CDK2', -0.290572696),
 ('CDK2', 'CDK5', -0.290572696),
 ('PIK3CB', 'PLK1', -0.064507194),
 ('LATS1', 'PRKCI', -0.156353307),
 ('ERN1', 'PLK1', -0.376956871),
 ('CAMKK2', 'TNK2', -0.113410283),
 ('CIT', 

### MEK/ERK communities contents

In [181]:
#Community containing MAP2K1 in the trametinib(MEK)[-] network (column 1 , negative z-scores)
#Community containing MAPK1_3 in the GDC0994(ERK)[-] network (column 4, negative z-scores)
MEK_ERKcom = [target_communities[0],target_communities[3]]
#Separate kinases in networks trametinib(MEK)[-] and GDC0994(ERK)[-] into:
#Kinases not present in the community of interest of that network
MEK_ERKnonc = [[x for x in negative_nodes[0] if x not in target_communities[0]],[x for x in negative_nodes[3] if x not in target_communities[3]]]
#Kinases present in both communities
MEK_ERKallc = [x for x in target_communities[0] if x in target_communities[3]]
MEK_ERKallc = [MEK_ERKallc,MEK_ERKallc]
#Kinases present in one of the communities, but not the other
MEK_ERKinc = [[x for x in target_communities[0] if x not in target_communities[3]],[x for x in target_communities[3] if x not in target_communities[0]]]


### PIK3CA/AKT communities contents

In [182]:
#Community containing PIK3CA in the GDC0941(PI3K)[-] network (column 2, negative z-scores)
#Community containing AKT1_2 in the AZD5363(AKT)[-] network (column 3, negative z-scores)
PI3K_AKTcom = [target_communities[1], target_communities[2]]
#Separate kinases in networks GDC0941(PI3K)[-] and AZD5363(AKT)[-] into:
#Kinases not present in the community of interest of that network
PI3K_AKTnonc = [[x for x in negative_nodes[1] if x not in target_communities[1]],[x for x in negative_nodes[2] if x not in target_communities[2]]]
#Kinases present in both communities
PI3K_AKTallc = [x for x in target_communities[1] if x in target_communities[2]]
PI3K_AKTallc = [PI3K_AKTallc, PI3K_AKTallc]
#Kinases present in one of the communities, but not the other
PI3K_AKTinc = [[x for x in target_communities[1] if x not in target_communities[2]],[x for x in target_communities[2] if x not in target_communities[1]]]

### Intersection and differences between communities

In [183]:
nonc = [MEK_ERKnonc[0],PI3K_AKTnonc[0],PI3K_AKTnonc[1],MEK_ERKnonc[1]]
allc = [MEK_ERKallc[0],PI3K_AKTallc[0],PI3K_AKTallc[1],MEK_ERKallc[1]]
inc = [MEK_ERKinc[0],PI3K_AKTinc[0],PI3K_AKTinc[1],MEK_ERKinc[1]]
        
kinase_groups = list(zip(nonc,allc,inc))
kinase_int = list(zip(allc,inc))

### Target of each treatment and the community identified containing that target 

In [184]:
#Show the community contents of the selected communities in networks trametinib(MAP2K1)[-], GDC0941(PI3K)[-],
#AZD5363(AKT)[-] and GDC0994(MAPK1_3), in that particular order
for n in range(len(target_communities)):
    print(treatments[n])
    print(target_communities[n])

MEK.Trametinib_vs_Control
['PIK3CA', 'MAP2K1', 'PRKACB', 'RPS6KA2', 'MAP3K1', 'MAPK1_3', 'MINK1', 'PIK3CB', 'AKT1_2', 'PDGFRB', 'PRKACA', 'PRKCI', 'YES1', 'CSNK1E', 'TTK', 'PLK1', 'TNK2', 'RPS6KB1', 'PAK3', 'PAK1', 'MTOR', 'LATS1', 'STK4']
PI3K.GDC0941_vs_Control
[['CIT', 'CSNK1E', 'PRKCI', 'ABL1', 'LIMK1_2', 'PAK1', 'PAK3', 'PLK1', 'PIK3CA', 'TTK', 'MAP3K1', 'MAPK14', 'LATS1', 'TNK2', 'MAP4K5', 'MINK1', 'RPS6KA2', 'SRPK1', 'MTOR', 'YES1', 'PDGFRB', 'MAP2K1', 'STK3', 'AKT1_2', 'PRKACA', 'PAK4', 'PIK3CB', 'PRKACB', 'MAPKAPK2', 'RPS6KB1']]
AKT.AZD5363_vs_Control
[['CIT', 'MAP4K5', 'CDK2', 'CSNK1E', 'MINK1', 'SRPK1', 'PRKCI', 'ABL1', 'LIMK1_2', 'PAK1', 'PAK3', 'PLK1', 'TTK', 'PIK3CA', 'MAPK14', 'PIK3CB', 'LATS1', 'TNK2', 'MAP3K1', 'MTOR', 'AKT1_2', 'PRKACA', 'PRKACB', 'MAPKAPK2', 'RPS6KB1']]
ERK.GDC0994_vs_Control
[['MAPK1_3', 'PDGFRB', 'SRPK3', 'CDK9', 'TNK2', 'ARAF', 'MAP2K1', 'PRKCI', 'ROCK1_2']]


## 2.2 - Network visualizations of networks trametinib(MAP2K1)[-], GDC0994(MAPK1_3), GDC0941(PI3K)[-], and AZD5363(AKT)[-]

In [23]:
for n in range(len(target_communities)):
    networkgraph = network_visualization(edges_list=negative_edges[n],kinases=kinase_groups[n],kg_colors=["#fcbb8b", "#857be3", "#baeeff"], treatment=treatments[n])
    

## 2.3 - Network visualization of intersection between the selected communities of  trametinib(MAP2K1)[-], GDC0941(PI3K)[-] and AZD5363(AKT)[-]

In [185]:
#Kinases present in the selected community of network trametinib(MAP2K1)[-], but not in the selected communities of 
#networks GDC0941(PIK3CA)[-] and AZD5363(AKT1_2)[-]
#MAP2K1only = [x for x in communities[0] if x not in communities[1] and x not in communities[2]]
#Kinases present accross the selected communities of networks trametinib(MAP2K1)[-], GDC0941(PIK3CA)[-] and AZD5363(AKT1_2)[-]
intersection = [x for x in target_communities[0] if x in target_communities[1] and x in target_communities[2]]
#Kinases present accross the selected communities of networks trametinib(MAP2K1)[-] and GDC0941(PIK3CA)[-], but not in the
#selected community of network AZD5363(AKT)[-]
MAP2K1_PIK3CA = [x for x in target_communities[0] if x in target_communities[1] and x not in target_communities[2]]
#Kinases present accross the selected communities of networks AZD5363(AKT)[-] and GDC0941(PIK3CA)[-] but not trametinib(MAP2K1)[-]
#and kinases present only in the selected communities of GDC0941(PIK3CA)[-] and AZD5363(AKT1_2)[-] networks
#therest = [[x for x in communities[3] if x in communities[2] and x not in communities[0]],[x for x in communities[3] if x not in communities[2] and x not in communities[0]], [x for x in communities[2] if x not in communities[3] and x not in communities[0]]]

kinase_groups = MAP2K1_PIK3CA, intersection
kinases = list(itertools.chain(*kinase_groups))
#Store kinase interactions across the three networks in which the two kinases are in any of the selected communities
#edges = negative_edges[0] + negative_edges[1] + negative_edges[2]
#edges = [(a,b,0) for a,b,c in edges if a in kinases and b in kinases]
edges = [ [(a,b) for a,b,c in negative_edges[n]] for n in [0,1,2] ]
edges = set(edges[0]).intersection(edges[1], edges[2])
edges = [(a,b) for a,b in edges if a in kinases and b in kinases]

#Intersection of communities visualized as a network
networkgraph= network_visualization(edges_list=edges,kinases=kinase_groups,kg_colors=["#fcbb8b", "#857be3", "#baeeff","green"], treatment='intersection_MAP2K1_PIK3CA_AKT')


## 2.4 - Community strength of the kinases in each of the selected communities of networks trametinib(MAP2K1)[-], GDC0941(PI3K)[-],AZD5363(AKT)[-] and GDC0994(MAPK1_3), visualized in heatmaps

In [200]:
treat_int=[0,3]
edges=negative_edges
communities=target_communities
treatments=treatments

#Calculate the community strength of each of the kinases in the two selected communities you are comparing
kin_cs = ['a'] * len(treat_int)

n=1
#for n in range(len(treat_int)):
na = treat_int[n]
# Store kinase pairs in which both kinases are in the community of interest
community_edges = [x for x in edges[na] if x[0] in communities[na] and x[1] in communities[na]]
# Calculate the community strength of each of the kinases in the community and store as dictionary
print(n)
#kin_cs[n] = {kin:abs(round(sum([community_edges[n][2] for n in range(len(community_edges)) if community_edges[n][1]==kin or community_edges[n][0]==kin]), 2)) for kin in communities[na]}  
kin_cs[n] = {kin:'x' for kin in communities[na]}

kin_cs


1


TypeError: unhashable type: 'list'

In [ ]:
treat_int=[0,3]
edges=negative_edges
communities=target_communities
treatments=treatments

#Calculate the community strength of each of the kinases in the two selected communities you are comparing
kin_cs = ['a'] * len(treat_int)
for n in range(len(treat_int)):
    na = treat_int[n]
    # Store kinase pairs in which both kinases are in the community of interest
    community_edges = [x for x in edges[na] if x[0] in communities[na] and x[1] in communities[na]]
    # Calculate the community strength of each of the kinases in the community and store as dictionary
    kin_cs[n] = {kin:abs(round(sum([community_edges[n][2] for n in range(len(community_edges)) if community_edges[n][1]==kin or community_edges[n][0]==kin]), 2)) for kin in communities[na]}    
    
#Combine two communities lists
kinases = list(dict.fromkeys(communities[treat_int[0]]+communities[treat_int[1]]))
#Get values for each of the kinases in the combined list in each of the communities
#If kinase is not in the community then the value is None
values = [[kin_cs[n][kin] if kin in communities[treat_int[n]] else None for kin in kinases]for n in [0,1]]
    
#Define heatmap axis and text
x = kinases
y = [ treatments[na]+'<br>community CS</br>' for na in treat_int]
z = [values[0],values[1]]



In [240]:
#trametinib(MAP2K1)[-] and GDC0994(MAPK1_3)
[x1,y1,z1] = cs_calculation(treat_int=[0,3], edges=negative_edges, communities=target_communities, treatments=treatments)

# HEATMAP
fig = go.Figure(data=go.Heatmap(
                z=z1,
                x=x1,
                y=y1,
                yaxis='y2',
                hoverongaps = False))

fig.update_layout(xaxis=dict(domain=[0.21,1]),
                  yaxis2=dict(anchor='free', position= 0, side='right'))


fig.show()


In [211]:
#GDC0941(PI3K)[-] and AZD5363(AKT)[-]
[x2,y2,z2] = cs_calculation(treat_int=[1,2], edges=negative_edges, communities=target_communities, treatments=treatments)

# HEATMAP
fig = go.Figure(data=go.Heatmap(
                z=z2,
                x=x2,
                y=y2,
                hoverongaps = False))
fig.show()

## 2.5 - Saving the edges and nodes of each community as dataframes

## Individual communities

In [170]:
#Save nodes and edges as dataframe to make nice visualizations in R

#nodes community strength
nodes_cs = list(zip([x1]+[x1],z1)) + list(zip([x2]+[x2],z2)) 
nodes_cs = [nodes_cs[0]] + [nodes_cs[2]] + [nodes_cs[3]] + [nodes_cs[1]]

#Filter to keep edges of interest only
negative_edges_com = [ [x for x in negative_edges[n] if (x[0] in list(itertools.chain(*kinase_int[n])) and x[1] in list(itertools.chain(*kinase_int[n])) ) ] for n in range(4)]
#Add weight to draw edges
negative_edges_pos = [ [ (*x , abs(x[2])*10 ) for x in network] for network in negative_edges_com ]

#Add community + color to nodes
nodes_inf = []
for n in range(len(kinase_int)):
    nodes_inf0 = list(zip( kinase_int[n][0], ['kinases present exclusively in this SC']*len(kinase_int[n][0]) ,  ['#96efff']* len(kinase_int[n][0]) )) 
    nodes_inf1 = list(zip(kinase_int[n][1], ['kinases present in both SCs']*len(kinase_int[n][1]) ,  ['#FFDB33']* len(kinase_int[n][1]) )) 
    nodes_inf_sum = nodes_inf0 + nodes_inf1
    nodes_df = pd.DataFrame( nodes_inf_sum, columns=['id', 'group', 'color'])
    #add community strength
    nodes_cs_df = pd.DataFrame( list(zip(nodes_cs[n][0], nodes_cs[n][1])) , columns = ['id','community_strength'] )
    nodes_df_add = pd.merge(nodes_df, nodes_cs_df, on='id')
    #save nodes as csv file
    nodes_df_add.to_csv('output_dataframes/{}_nodes.csv'.format(treatments[n]))
    #save edges as csv file
    edges_df = pd.DataFrame(negative_edges_pos[n], columns=['from','to','z-score','weight'])
    edges_df.to_csv('output_dataframes/{}_edges.csv'.format(treatments[n]))



In [ ]:
## Intersection of trametinib, GDC0941 and AZD5363

In [171]:
#Calculate average edge weights across the three networks
edges_MAP2K1_PIK3CA_AKT = [ { (edge[0],edge[1]):abs(edge[2]) for edge in negative_edges[n]} for n in [0,1,2]  ]   
edges_MAP2K1_PIK3CA_AKT_sum = dict( functools.reduce(operator.add, map(collections.Counter, xx))  )
edges_MAP2K1_PIK3CA_AKT_ave = [ (*key, (result[key]/3)) for key in edges ]

#Save nodes and edges as dataframe to make nice network visualizations in R
edges_df = pd.DataFrame(edges_MAP2K1_PIK3CA_AKT_ave, columns=['from','to','weight'])
edges_df.to_csv('output_dataframes/MAP2K1_PIK3CA_AKT_edges.csv')

intersection_x = list(zip(intersection, ['MAP2K1_PIK3CA_AKT'] * len(intersection), ['#96efff']* len(intersection) ) )
MAP2K1_PIK3CA_x = list(zip(MAP2K1_PIK3CA, ['MAP2K1_PIK3CA_notAKT']*len(MAP2K1_PIK3CA), ['#FFDB33']* len(MAP2K1_PIK3CA) ) )
nodes = intersection_x + MAP2K1_PIK3CA_x
nodes_df = pd.DataFrame( nodes, columns=['id', 'group', 'color'])

nodes_df.to_csv('output_dataframes/MAP2K1_PIK3CA_AKT_nodes.csv')

# Positive edges

In [70]:
#Store dataset treatments names, kinase interactions (network edges) with a z-score < 0, and their z-scores (edges weight) 
[treatments,positive_edges, positive_nodes] = dataset_info('input/PROJECT_DATASET_2.csv', sign = 'pos')

In [ ]:
#Store contents of communities identified in each network
all_pos_communities = ['a'] * 4
for n in list(range(0,4)):
    [A_asso,n_iter,labels, kin_communities, target_community] = converge_asso(positive_edges[n])
    #All identified communities
    all_pos_communities[n] = kin_communities

In [79]:
n=0
print(targets[n])
all_pos_communities[n]

MAP2K1


[['ARAF',
  'ERN1',
  'CAMKK2',
  'CIT',
  'TAOK3',
  'IRAK1',
  'MAP4K5',
  'MAPK14',
  'PDGFRB',
  'CDK2',
  'CSNK1E',
  'MAP3K1',
  'MAP4K4',
  'YES1',
  'MAPK1_3',
  'MINK1',
  'SRPK1',
  'PRKCI',
  'SRPK3',
  'KIT',
  'MAPK9',
  'PLK1',
  'RPS6KA2',
  'CDK4',
  'TTK',
  'PAK3',
  'ROCK1_2'],
 ['CDK9', 'ABL2', 'FYN', 'MELK', 'CDK17', 'ULK1', 'CDK6', 'SIK2'],
 ['PIK3CA', 'ABL1', 'AKT1_2', 'MAPKAPK2'],
 ['TNK2', 'MAP3K3', 'CSNK2A2'],
 ['PRKACA', 'LATS1', 'LIMK1_2'],
 ['PAK1', 'PAK4', 'RPS6KA4'],
 ['STK4', 'PIK3CB'],
 ['CDKL5'],
 ['CAMK2D'],
 ['PKN2'],
 ['CLK1'],
 ['MAP2K1'],
 ['CDK1'],
 ['CAMK2A'],
 ['RPS6KA6'],
 ['MAST1'],
 ['STK3'],
 ['PHKG2'],
 ['MARK3'],
 ['RPS6KA3'],
 ['CDK5']]

# 3 - Example 2 - Analysis and Results

# Negative edges

In [118]:
#Store dataset treatments names, kinase interactions (network edges) with a z-score < 0, and their z-scores (edges weight) 
[treatments2,negative_edges2,negative_nodes2] = dataset_info('input/example2_edges_zscores.csv')
#Main kinase targets of each of the treatments/inhibitors in the dataset
targets2 = ['MTOR']

In [148]:
#Store contents of communities identified in each network
[A_asso,n_iter,labels, kin_communities, target_community] = converge_asso(edges_list=negative_edges2[0], target=targets2[0])
#All identified communities
all_communities2 = kin_communities
#Because we noticed that when extracting the community of interest with the converge_asso
#function the contents of said community differ from run from run, for this network we run the community detection function (converge_asso)
#50 times, and keep only the nodes that are present in the community in all 50 runs of the converge_asso function
target_communities2 = rep_check(50, negative_edges2[0], targets2[0])

target_communities2

['MAPK1_3',
 'SRPK3',
 'PDGFRB',
 'LIMK1_2',
 'TTK',
 'MTOR',
 'ROCK1_2',
 'TNK2',
 'PAK3',
 'SRPK1',
 'AKT1_2',
 'ABL1']

# Positive edges

In [149]:
#Store dataset treatments names, kinase interactions (network edges) with a z-score < 0, and their z-scores (edges weight) 
[treatments2,positive_edges2, positive_nodes2] = dataset_info('input/example2_edges_zscores.csv', sign='pos')


In [150]:
#Store contents of communities identified in each network
[A_asso,n_iter,labels, kin_communities, target_community] = converge_asso(positive_edges2[0], target=targets2[0])
#All identified communities
pos_all_communities2 = kin_communities
#Community containing the target
pos_target_communities2 = target_community

pos_target_communities2

[['ERN1',
  'PDGFRB',
  'MAPK1_3',
  'PRKCI',
  'PLK1',
  'TNK2',
  'AKT1_2',
  'PIK3CA',
  'MAP2K1',
  'STK3',
  'SRPK3',
  'MTOR',
  'STK4',
  'HIPK3',
  'MAP3K2',
  'CDK4',
  'MAPKAPK2',
  'RPS6KB1']]

# Comparisson of MTOR community in negative and positive edges

In [154]:
neg_pos_target_communities = [target_communities2, pos_target_communities2[0]]

neg_pos_target_communities 

In [ ]:
[x1,y1,z1] = cs_calculation(treat_int=[0,1], communities=neg_pos_target_communities, treatments=community_names)

# HEATMAP
fig = go.Figure(data=go.Heatmap(
                z=z1,
                x=x1,
                y=y1,
                hoverongaps = False))
fig.show()